In [ ]:
ROC_by_HSA_full = {}
accuracy_by_HSA_full = {}
norm_MCC_by_HSA_full = {}

weeks_to_predict = range(1, 121 - 1 - 3 - 1)




for prediction_week in weeks_to_predict:
    print(prediction_week)
    ROC_by_week_full = {}

    accuracy_by_week_full = {}
    MCC_by_week_full = {}
    for HSA in HSA_weekly_data_all['HSA_ID'].unique():
        training_dataframe, testing_dataframe = LOOCV_by_HSA_dataset(HSA_weekly_data_all, HSA, 'HSA_ID')
        if not testing_dataframe['weight'].isna().any():
            model_name_to_load = "Full_model_" + time_period + "_" + str(prediction_week) + ".sav"
            clf_full_period = pickle.load(open(model_name_to_load, 'rb'))
            #period    

            X_train_full, y_train_full, weights_full, missing_data_train_HSA = prep_training_test_data(training_dataframe, no_weeks=range(1, int(prediction_week + train_weeks_for_initial_model) + 1), weeks_in_future=3, geography='HSA_ID', weight_col='weight', keep_output=True)

            X_test_full, y_test_full, weights_test_full, missing_data_test_HSA = prep_training_test_data_period(testing_dataframe, no_weeks=range(int(prediction_week + train_weeks_for_initial_model) + 1, int(prediction_week + train_weeks_for_initial_model + size_of_test_dataset) + 1), weeks_in_future=3, geography='HSA_ID', weight_col='weight', keep_output=True)
            if len(X_test_full) >= 1:
                weights_full = weights_full[0].to_numpy()
                clf_full_period.fit(X_train_full, y_train_full, sample_weight=weights_full)

                # Make predictions on the test set
                y_pred = clf_full_period.predict(X_test_full)
                y_pred_proba = clf_full_period.predict_proba(X_test_full)

                # Evaluate the predictive power of the model
                conf_matrix = confusion_matrix(y_test_full, y_pred)

                accuracy_by_week_full[HSA] = accuracy_score(y_test_full, y_pred)
                if (sum(y_test_full[0]) == len(y_test_full[0])) | (sum(y_test_full[0]) == 0):
                    ROC_by_week_full[HSA] = None
                    MCC_by_week_full[HSA] = None
                else:
                    ROC_by_week_full[HSA] = roc_auc_score(y_test_full, y_pred_proba[:, 1])
                    MCC_by_week_full[HSA] = (matthews_corrcoef(y_test_full, y_pred) + 1)/2




    ROC_by_HSA_full[prediction_week] = ROC_by_week_full
    accuracy_by_HSA_full[prediction_week] = accuracy_by_week_full
    norm_MCC_by_HSA_full[prediction_week] = MCC_by_week_full

In [ ]:
ALL_METRICS_FULL = [ROC_by_HSA_full, accuracy_by_HSA_full, norm_MCC_by_HSA_full]
ylabels = ['auROC', 'Accuracy', 'MCC']
labels_subplots = ['A', 'B', 'C', 'D', 'E', 'F']

In [ ]:
weeks_to_predict = range(1,91)
for j, metric in enumerate(ALL_METRICS_FULL):
    fig, axs = plt.subplots(figsize=(10, 6), facecolor='w', edgecolor='k')

    all_values = []
    labels = []
    labels_ticks = [0]
    for i, prediction_week in enumerate(weeks_to_predict):
        values = metric[prediction_week].values()
        values = [float(value) for value in values if value != 'NA' and value is not None]  # Convert and filter
        
        if values:
            all_values.append(values)
            labels.append(f'Week {prediction_week}')
        if prediction_week % 5 == 0:  # Label every 5th week
                labels_ticks.append(f'{prediction_week}')
        
    # Create a combined violin and box plot
    whisker_props = dict(linewidth=1, color='#929BA0')
    box_props = dict(facecolor='#A7BBEC', edgecolor='#929BA0')
    outlier_props = dict(marker='.', markersize=2, markerfacecolor='#929BA0', markeredgecolor='#929BA0')

    axs.boxplot(all_values, labels=labels, showfliers=True, patch_artist=True,boxprops=box_props, whiskerprops=whisker_props, flierprops = outlier_props)
    # Rotate x-axis labels for better readability
    plt.xticks(rotation=45, ha='right')

    # Set ticks only for every 5th week
    axs.set_xticks(np.arange(0, 91, step=5))
    axs.set_xticklabels(labels_ticks, rotation=0)

    # Add labels and title
    plt.xlabel('Weeks after 2020-07-15', fontsize=18)
    plt.ylabel(ylabels[j], fontsize=18)
    axs.text(
            axs.get_xlim()[0],
            axs.get_ylim()[1] * 1.01,
            labels_subplots[j],
            fontsize=24,
            **hfont,
        )
    plt.tight_layout()